# Mini-Projeto MLP-CNN
## Centro de Informática da UFPE
### IF702 - Redes Neurais
### Germano Crispim Vasconcelos

14/04/2025

Grupo:
1. Pedro Lima (pbsl@cin.ufpe.br)
2. Pedro de Souza (pbsl@cin.ufpe.br)
3. Pedro Barros (pbsl@cin.ufpe.br)


- Conjunto de classificadores disponíveis:
    - Perceptron multicamadas (MLP)
    - Kolmogorov Arnold Networks (KANs).
    - Modelo Baseado em Transformer (STab) 2024 (em preparação)
    - KAN Transformer (TabKANet) 2024 (em preparação)
    - Random Forest (usado para comparação)
    - Gradient Boosting (usado para comparação)

- Investigar diferentes topologias da rede e diferentes valores
    dos parâmetros (básico):
    - Número de camadas
    - Número de unidades intermediárias
    - Variação da taxa de aprendizagem
    - Função de ativação (logistica, tangent hiperbolica, Relu)
    - Otimização: Adam, Drop-out, Regularização
    - Usar método de amostragem básica (repetitive oversampling)

- Parâmetros adicionais que podem ser explorados:
    - Algoritmo de aprendizagem
    - Taxa de aprendizagem adaptativa
    - Outros

### Preparação de dados (divisão e balanceamento)
- Conjuntos de dados
- Treinamento
    - Validação (separar amostra do Treinamento)
    - Teste (separar amostra do Treinamento)
- Estatisticamente representativos e independentes
    - Nenhuma informação do conjunto de teste pode interferir nos conjuntos de treinamento e validação (ex: identificação do mínimo e máximo para normalização). (vazamento de dados)
    - Não pode haver sobreposição (contaminação)

### Avaliação de Desempenho
- Classificação
    - Teste estatístico Kolmogorov-Smirnov 
        - KS (principal) 
        - MSE (erro médio quadrado)
    - Matriz de confusão
    - Auroc (Área sob a Curva Roc)
    - Recall, Precision e F-Measure

### Experimentos
- Pré-processamento da base de dados
    - Tratamento de dados ausentes, se houver (missing data)
    - Remoção de ruídos (outliers), se houver
    - Remoção de inconsistências, se houver
    - Normalização
    - Codificação
    - Transformação de variáveis
    - Criação de variáveis agregadas
- Importante
    - Registrar o desempenho de forma evolutiva, a cada etapa. Não
    elimine váriáveis no primeiro modelo (a não ser identificadores)

- Recomendação:
    - Iniciar com um modelo MLP e um modelo
    Random Forest
    - Após bom desempenho com esses modelos,
    experimentar os demais
    - KANs, Transformer (tentativo), KAN
    Transformer (tentativo)
    - Gradient boosting.

### Ferramentas para o projeto

- Código em Python
    - https://github.com/RomeroBarata/IF702-redes-neurais

- Pode usar qualquer biblioteca, preocupando-se apenas de garantir que está executando corretamente os experimentos e análise de performance (exemplo, usar função do KS que meça corretamente os valores, comparar com os gráficos dos slides neste ppt)
- Conjuntos de dados do problema
    - Arquivo obtido do Kaggle

### Lições aprendidas

- Comece com uma rede pequena: 1 camada, 10 unidades (a melhor rede é a menor rede que resolve bem o problema: navalha de Occam)
- Definir numero de epocas maximo em 10mil! Usar o critério de parada baseado no Patience (Max Fail = 20)
- Taxas de aprendizagem menores requerem mais tempo mas tendem a gerar melhores resultados
- Fazer backup automático
- Começar cedo, se deixar para ultimo mês, não vai sair!
- Considerar Optuna pode ser uma boa estratégia, caso contrário use gridsearch

### Resultados do projeto

- Apresentação com todos do grupo com estrutura experimental e interpretação dos resultados
- Entrega no final do semestre (PPT e código)

### Dicionário do Dataset: 

Atributos de entrada:
- gender:	Customer's gender (Male/Female)
- SeniorCitizen:	Indicates if the customer is a senior citizen (1 = Yes, 0 = No)
- Partner:	Whether the customer has a partner (Yes/No)
- Dependents:	Whether the customer has dependents (Yes/No)
- tenure:	Number of months the customer has stayed with the company
- PhoneService:	Whether the customer has a phone service (Yes/No)
- MultipleLines:	Whether the customer has multiple phone lines (No, Yes, No phone service)
- InternetService:	Type of internet service (DSL, Fiber optic, No)
- OnlineSecurity:	Whether the customer has online security (Yes, No, No internet service)
- OnlineBackup:	Whether the customer has online backup (Yes, No, No internet service)
- DeviceProtection:	Whether the customer has device protection (Yes, No, No internet service)
- TechSupport:	Whether the customer has tech support (Yes, No, No internet service)
- StreamingTV:	Whether the customer has streaming TV (Yes, No, No internet service)
- StreamingMovies:	Whether the customer has streaming movies (Yes, No, No internet service)
- Contract:	Type of contract (Month-to-month, One year, Two year)
- PaperlessBilling:	Whether the customer has paperless billing (Yes/No)
- PaymentMethod:	Payment method used (Electronic check, Mailed check, Bank transfer, Credit card)
- MonthlyCharges:	Monthly charges the customer pays
- TotalCharges:	Total amount charged to the customer

Atributo alvo:
- Churn:	Whether the customer has churned (Yes/No)

# Imports e Funções

In [ ]:
# Dataset
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
import seaborn as sns
#import tensorflow as tf
import torch
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
#from math import log2
from random import randint
%matplotlib inline

In [ ]:
# Modelos
from sklearn.neural_network import MLPClassifier
from kan import KAN, KANClassifier # KANClassifier é uma classe feita por mim, da meu fork da biblioteca do pykan no github
# [STab]
# [KAN Transformer]
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
'''import xgboost as xgblib
from xgboost import XGBClassifier'''

# Ferramentas de ML
#import joblib
import mlflow
import mlflow.sklearn
import logging
from sklearn.model_selection import GridSearchCV

# Avaliação
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, average_precision_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import cross_val_score

# Explicabilidade
import matplotlib.pyplot as plt
import scikitplot as skplt
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay as pdp
from lime import lime_tabular
%matplotlib inline

Funções tiradas dos scripts disponibilizados:

In [ ]:
def extract_final_losses(history):
    """Função para extrair o melhor loss de treino e validação.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    Dicionário contendo o melhor loss de treino e de validação baseado 
    no menor loss de validação.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    idx_min_val_loss = np.argmin(val_loss)
    return {'train_loss': train_loss[idx_min_val_loss], 'val_loss': val_loss[idx_min_val_loss]}

def plot_training_error_curves(history):
    """Função para plotar as curvas de erro do treinamento da rede neural.
    
    Argumento(s):
    history -- Objeto retornado pela função fit do keras.
    
    Retorno:
    A função gera o gráfico do treino da rede e retorna None.
    """
    train_loss = history.history['loss']
    val_loss = history.history['val_loss']
    
    fig, ax = plt.subplots()
    ax.plot(train_loss, label='Train')
    ax.plot(val_loss, label='Validation')
    ax.set(title='Training and Validation Error Curves', xlabel='Epochs', ylabel='Loss (MSE)')
    ax.legend()
    plt.show()

def compute_performance_metrics(y, y_pred_class, y_pred_scores=None):
    accuracy = accuracy_score(y, y_pred_class)
    recall = recall_score(y, y_pred_class)
    precision = precision_score(y, y_pred_class)
    f1 = f1_score(y, y_pred_class)
    performance_metrics = (accuracy, recall, precision, f1)
    if y_pred_scores is not None:
        skplt.metrics.plot_ks_statistic(y, y_pred_scores)
        plt.show()
        y_pred_scores = y_pred_scores[:, 1]
        auroc = roc_auc_score(y, y_pred_scores)
        aupr = average_precision_score(y, y_pred_scores)
        performance_metrics = performance_metrics + (auroc, aupr)
    return performance_metrics

def print_metrics_summary(accuracy, recall, precision, f1, auroc=None, aupr=None):
    print()
    print("{metric:<18}{value:.4f}".format(metric="Accuracy:", value=accuracy))
    print("{metric:<18}{value:.4f}".format(metric="Recall:", value=recall))
    print("{metric:<18}{value:.4f}".format(metric="Precision:", value=precision))
    print("{metric:<18}{value:.4f}".format(metric="F1:", value=f1))
    if auroc is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUROC:", value=auroc))
    if aupr is not None:
        print("{metric:<18}{value:.4f}".format(metric="AUPR:", value=aupr))

Funções minhas:

In [ ]:
def pfi(model, x, y, name=None):
    result = permutation_importance(model, x, y,n_repeats=30, random_state=0)

    #cols = [f"[{i}] - {x.columns[i]}" for i in range(len(x.columns))]

    cols = ['[0] sex', '[1] age', '[2] creat.plsm.creat', '[3] dim.d.p.d.d.quant', '[4] hemgrm.eos.%',
        '[5] hemgrm.hemgb', '[6] hemgrm.leuc', '[7] hemgrm.linf.%', '[8] hemgrm.mono.%',
        '[9] hemgrm.neut.%', '[10] hemgrm.plaq', '[11] pot.pot', '[12] prot.c.r.plsm', '[13] sod.sod',
        '[14] tgo.ast.tgo', '[15] tpg.alt.tgp', '[16] ureia.plsm.ureia']

    importances = pd.Series(result.importances_mean, index=cols)

    fig, ax = plt.subplots()

    importances.plot.bar(yerr=result.importances_std, ax=ax)

    if(name):
        ax.set_title(f"Feature importances on {name} model")
    else:
        name = str(model)
        i = name.find('(')
        ax.set_title(f"Feature importances on {name[:i]} model")
    ax.set_ylabel("Mean performance decrease")
    fig.tight_layout()
    plt.show()

# Dataset

In [10]:
df = pd.read_csv('customer_churn_telecom_services.csv', header=0)
print('Nº de instâncias:', len(df))
df.head()

Nº de instâncias: 7043


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [9]:
# Tipos das colunas
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [11]:
# Quantiade de valores únicos por coluna
df.nunique()

gender                 2
SeniorCitizen          2
Partner                2
Dependents             2
tenure                73
PhoneService           2
MultipleLines          3
InternetService        3
OnlineSecurity         3
OnlineBackup           3
DeviceProtection       3
TechSupport            3
StreamingTV            3
StreamingMovies        3
Contract               3
PaperlessBilling       2
PaymentMethod          4
MonthlyCharges      1585
TotalCharges        6530
Churn                  2
dtype: int64

In [36]:
# Quantiades de cada valor único por coluna
for col in df.columns:
    unique_values = df[col].value_counts()
    if(len(unique_values) <= 4):
        print(unique_values, '\n')

gender
Male      3555
Female    3488
Name: count, dtype: int64 

SeniorCitizen
0    5901
1    1142
Name: count, dtype: int64 

Partner
No     3641
Yes    3402
Name: count, dtype: int64 

Dependents
No     4933
Yes    2110
Name: count, dtype: int64 

PhoneService
Yes    6361
No      682
Name: count, dtype: int64 

MultipleLines
No                  3390
Yes                 2971
No phone service     682
Name: count, dtype: int64 

InternetService
Fiber optic    3096
DSL            2421
No             1526
Name: count, dtype: int64 

OnlineSecurity
No                     3498
Yes                    2019
No internet service    1526
Name: count, dtype: int64 

OnlineBackup
No                     3088
Yes                    2429
No internet service    1526
Name: count, dtype: int64 

DeviceProtection
No                     3095
Yes                    2422
No internet service    1526
Name: count, dtype: int64 

TechSupport
No                     3473
Yes                    2044
No internet ser

Pelas quantidades de aparições de cada valor nas colunas, se fizermos um shuffling ao dividir o dataset em treino e teste, provavelmente não teremos problemas.

In [ ]:
# Procurando valores nulos
df.isna().any()

gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
OnlineBackup        False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
PaymentMethod       False
MonthlyCharges      False
TotalCharges         True
Churn               False
dtype: bool

Apenas a coluna TotalCharges (valor total cobrado ao cliente) possui valores nulos.

In [38]:
# Número de valores nulos
df['TotalCharges'].isna().sum()

11

Pela quantidade, pode-se usar um KNNImputer ou trocar pelo valor mediano da coluna.

## Divisão do dataset

In [ ]:
seed = randint(0, 4294967295)
df_train, df_test = train_test_split(df, test_size=0.25, shuffle=True, random_state=seed)
#df_train, df_val = train_test_split(df_train, test_size=1/3, shuffle=True, random_state=seed)
print(seed)

# Modelos de ML
Treinamento, teste e avaliações

### Multi-Layer Perceptron

- hidden_layer_sizes (estrura da rede)
- activation (função de ativação)
- solver* (algoritmo de aprendizagem/otimizador)
- learning_rate_init (taxa de aprendizado)
- learning_rate* (taxa de aprendizado fixa ou adaptativa)
- alpha (regularização L1)
- max_iter = 10000 (número máximo de épocas)
- early_stopping = True (usar early stopping)
- n_iter_no_change = 20 (paciência)
- shuffle = True
- random_state = seed

* = opcional

In [ ]:
mlp = MLPClassifier()#.fit(X_train, y_train)

### Kolmogorov-Arnold Network

- width (estrutura da rede)
- grid (número de intervalos do grid)
- k (ordem do spline)
- random_state = seed
- lr (taxa de aprendizado)
- steps (número de épocas)
- opt* (algoritmo de aprendizagem/otimizador)

* = opcional

In [ ]:
kanet = KANClassifier()#.fit(X_train, y_train)

### STab

### TabKANet

### Random Forest

In [ ]:
rforest = RandomForestClassifier()#.fit(X_train, y_train)

In [ ]:
RandomForestClassifier().fit()

### Grandient Boosting

In [ ]:
gboost = GradientBoostingClassifier()#.fit(X_train, y_train)

#### XGBoost

In [ ]:
#xgb = XGBClassifier.fit(X_train, y_train)

#### Caso queira usar a biblioteca da KAN (pykan) original:

In [ ]:
'''from kan import *
from sklearn.base import BaseEstimator
from sklearn.utils.validation import check_is_fitted

# Função de perda:
class kanLoss(torch.nn.CrossEntropyLoss):
  def forward(self, input, target):
        return super().forward(input.type(torch.float64), target.type(torch.long))

# Classe do modelo KAN: (TODO: Maybe change the declare args so the base KAN model is declared inside the KANClassifier?)
class KANClassifier(BaseEstimator):
    #Classe para o modelo KAN, para que se pareça mais com outras classes de
    #modelos de AM do scikit-learn

    #Parâmetros:
    #- model: modelo original da classe MultKAN() | KAN()

    #Exp.: KANnet(KAN(width=[2,2], grid=3, k=3))
    
    def __init__(self, model:KAN) -> None:
       self.model = model
       self._estimator_type = "classifier"
       self.data = {}
       self.results = {}
       self.accuracy = None
       self.precision = None
       self.recall = None

    def get_params(self, deep=False):
        # suppose this estimator has parameters "alpha" and "recursive"
        return {"width": self.model.width, "grid": self.model.grid, "k": self.model.k}

    # Transformação de dataset:
    def __dt4kan(self, datarray):
        if isinstance(datarray, np.ndarray):
            return torch.from_numpy(datarray).float()
        elif isinstance(datarray, torch.Tensor):
            return datarray
        return torch.from_numpy(np.array(datarray)).float()

    # Métricas
    def train_acc(self):
        return torch.mean((torch.argmax(self.model(self.data['train_input']), dim=1) == self.data['train_label']).float())

    def test_acc(self):
        return torch.mean((torch.argmax(self.model(self.data['test_input']), dim=1) == self.data['test_label']).float())

    def test_prec(self, lbl = 1):
        p_hat = (torch.argmax(torch.softmax(self.model(self.data['test_input']), dim=1), dim=1) == lbl)
        vp = (p_hat & (self.data['test_label'].float() == lbl))
        return (vp.sum()/p_hat.sum()).float()

    def test_recall(self):
        p = (self.data['test_label'] == 1)
        vp = (p & (torch.argmax(torch.softmax(self.model(self.data['test_input']), dim=1), dim=1) == 1))
        return (vp.sum()/p.sum()).float()

    # Fit
    # TODO: 
    # - Include all arguments of the original fit function from MultKAN
    # - Add possibility of adding other metrics (?)
    def fit(self, dataset:dict, opt="LBFGS", steps:int=20, loss_fn=kanLoss()):
        #Função de treinamento do modelo KAN

        #Parâmetros:
        #- dataset: <dict[Tensor]> que deve ter as seguintes chaves:
        #    - "train_input"
        #    - "train_label"
        #    - "test_input"
        #    - "test_label"

        #- opt: ...
        #- steps: ...
        #- loss_fn: ...
        
        self.is_fitted_ = True
        self.data = dataset
        self.classes_ = self.data['train_label'].unique()

        self.results = self.model.fit(self.data,
                                      opt=opt,
                                      steps=steps,
                                      metrics=(self.train_acc,
                                               self.test_acc,
                                               self.test_prec,
                                               self.test_recall),
                                      loss_fn=loss_fn)
        self.accuracy, self.precision, self.recall = self.results['test_acc'][-1], self.results['test_prec'][-1], self.results['test_recall'][-1]
        self.classes_ = np.array([i for i in range(self.predict_proba(self.data['test_input'][:2]).shape[1])])
        return self

    # Predições:
    def predict(self, new_data:torch.Tensor | np.ndarray) -> np.ndarray:
        sklearn.utils.validation.check_is_fitted(self)
        new_data = self.__dt4kan(new_data)
        return torch.argmax(torch.softmax(self.model(new_data), dim=1), dim=1).detach().numpy()

    def predict_proba(self, new_data:torch.Tensor | np.ndarray) -> np.ndarray:
        new_data = self.__dt4kan(new_data)
        return torch.softmax(self.model(new_data), dim=1).detach().numpy()

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))

    # Plotting:
    def plot(self, beta=100):
        self.model.plot(beta=beta) # não sei para que serve o beta

    # TODO: Change this function to return the figure, instead of just showing it right away
    def plot_metric(self, metric:str):
        if(metric not in ('loss', 'train_loss', 'test_loss', 'acc', 'train_acc', 'test_acc', 'test_prec', 'test_recall')):
            raise ValueError(f"'{metric}' isn't a valid plottable metrics, which are: 'loss', 'train_loss', 'test_loss', 'acc', 'train_acc', 'test_acc', 'test_prec', 'test_recall'")

        if((metric == 'loss') or (metric == 'acc')):
            plot0 = [float(x) for x in self.results['train_'+metric]]
            plot1 = [float(x) for x in self.results['test_'+metric]]
            sns.lineplot(y=plot0, x=range(1, len(plot0) + 1))
        else:
            plot1 = [float(x) for x in self.results[metric]]
        sns.lineplot(y=plot1, x=range(1, len(plot1) + 1))

        title = metric
        if(title[-3:] == 'acc'):
            title += 'uracy'
        if(title[0] == 't'):
            if(title[-4:] == 'prec'):
                title += 'ision'
            i = title.index('_')
            title = f'{title[i+1:]} ({title[:i]})'
        plt.title(title.capitalize())

        plt.show()
'''